# 可変長位置引数を使って、見た目をすっきりさせる

In [3]:
def log(message, values):
  if not values:
    print(message)
  else:
    values_str = ','.join(str(x) for x in values)
    print(f'{message}: {values_str}')

In [5]:
log('My numbers are', [1, 2])
log('Hi there', []) # 空のリストを渡さないといけない

My numbers are: 1,2
Hi there


In [1]:
# * (スター引数)を使うと見た目をすっきりさせられる
def log(message, *values):
  if not values:
    print(message)
  else:
    values_str = ','.join(str(x) for x in values)
    print(f'{message}: {values_str}')

In [2]:
log('My numbers are', 1, 2)
log('Hi there')

My numbers are: 1,2
Hi there


ただし、可変個数位置引数を使うには、注意が必要！

*args で受け取った引数はタプルに評価されるという性質が、一部のケースでは問題になる可能性がある

In [ ]:
# 悪い例
def my_generator():
  for i in range(10):
    print(f'yield: {i}')
    yield i

def my_func(*args): # *args は、タプルになる
  print(f'args: {args}')
  for arg in args:
    print(arg)

it = my_generator()
my_func(*it) # ここで全部消費される

yield: 0
yield: 1
yield: 2
yield: 3
yield: 4
yield: 5
yield: 6
yield: 7
yield: 8
yield: 9
args: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
0
1
2
3
4
5
6
7
8
9


In [ ]:
# 良い例
def my_generator():
  for i in range(10):
    print(f'yield: {i}')
    yield i

def my_func(args): # 1つのイテラブルとして受け取る
  print(f'args: {args}') # generator object が表示される
  for arg in args: # 1つずつ消費
    print(arg)

it = my_generator()
my_func(it) # ここではまだジェネレータとして渡される


args: <generator object my_generator at 0x7f26292b2c00>
yield: 0
0
yield: 1
1
yield: 2
2
yield: 3
3
yield: 4
4
yield: 5
5
yield: 6
6
yield: 7
7
yield: 8
8
yield: 9
9


これの何が問題か

## 例: 巨大なログファイルを読むジェネレータ

In [ ]:
# この関数は、巨大なログファイル（何GBもあるかも）を1行ずつ読み込むジェネレータ
# メモリ節約のため、全行を一度に読み込まないのがポイント。
def read_log_lines(filepath):
    with open(filepath) as f:
        for line in f:
            yield line.strip()

In [ ]:
# 間違った例
def process_lines(*lines): # *lines で渡してしまったとする
    for line in lines:
        print(f"Processing: {line}")

log_lines = read_log_lines("huge.log") # 仮なので、実行するとエラー
process_lines(*log_lines)

問題点：
* *log_lines の時点で、全行を読み込んでタプルにしてから関数に渡してしまう
* メモリ爆食い & パフォーマンス最悪（最悪メモリクラッシュ）

In [ ]:
# 正しい例
def process_lines(lines):  # タプルじゃなく、イテラブルとして受け取る
    for line in lines:
        print(f"Processing: {line}")

log_lines = read_log_lines("huge.log")
process_lines(log_lines)  # OK！行ごとに1つずつ処理される

メリット:
* メモリ効率◎
* 1行ずつ処理され、不要になったら途中で止めることもできる

補足:
* ジェネレータは、「遅延評価」の特性があるが、* を使うとその「遅延評価」が崩れてしまう（全部先に評価されてしまう）